In [1]:
%load_ext autoreload
%autoreload 2

In [22]:
import os, json
import numpy as np

if not os.getcwd().endswith("src"):
    os.chdir("src")
    print("jumping into src")

from utils.data.data_module import DataModule
from utils.data.testbench import TestBench
from recommender.run_pipeline import Models

if os.getcwd().endswith("src"):
    os.chdir("..")
    print("jumping out of src")

args = {
    "output_dir": "models/popularity",
    "dataset_config": "configs/datasets/id_dataset.json",
    "model_config": "configs/twotower/user_embedder.json",
    "model": "tower",
    "should_return_ids": True,
}
dataset_config = {}
model_config = {}
output_dir = args["output_dir"]
if args["dataset_config"]:
    with open(args["dataset_config"], "r") as f:
        dataset_config_2 = json.load(f)
        dataset_config.update(dataset_config_2)
if args["model_config"]:
    with open(args["model_config"], "r") as f:
        model_config = json.load(f)

os.makedirs(output_dir, exist_ok=True)
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
os.environ["TORCH_USE_CUDA_DSA"] = "1"

jumping into src
jumping out of src


In [3]:
datamodule = DataModule(**dataset_config)
testbench = TestBench(
    datamodule, should_return_ids=args.get("should_return_ids", False)
)

Resetting Train to k=0 ...: 100%|██████████| 48669/48669 [00:06<00:00, 7323.07it/s]

Number of Users: 54077, Hash[:8]: 9f0cd3, Hash: 9f0cd3119bd9ee7279856737c33aebb8
Total Animes: 12294, Total Users: 54077


In [26]:
auxiliary_args = {
    "n_users": datamodule.max_user_count,
    "n_anime": datamodule.max_anime_count,
}
model_config = model_config | auxiliary_args
model = Models.from_string(args["model"].upper())(datamodule=datamodule, **model_config)

model.train()

metrics = testbench.full_evaluation(model)
with open(os.path.join(output_dir, "output.txt"), "w") as f:
    for k, v in metrics.items():
        if type(v) == np.ndarray:
            continue
        f.write(f"{k}: {v}\n")

Epoch 1 / 20


L:2.291, +:-0.028, -:-0.044: 100%|██████████| 212/212 [00:13<00:00, 15.46it/s]


[0] Loss: 2.3137708666189667
Epoch 2 / 20


L:2.206, +:-0.012, -:-0.007: 100%|██████████| 212/212 [00:13<00:00, 15.89it/s]


[1] Loss: 2.2231271750522117
Epoch 3 / 20


L:2.165, +:-0.023, -:-0.006:  29%|██▉       | 61/212 [00:03<00:09, 15.92it/s]


KeyboardInterrupt: 